In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [3]:
path = "/FileStore/tables/Youtuber_log_file"
user_log = spark.read.json(path)

In [4]:
user_log.printSchema()

root
-- auth: string (nullable = true)
-- firstName: string (nullable = true)
-- gender: string (nullable = true)
-- itemInSession: long (nullable = true)
-- lastName: string (nullable = true)
-- length: double (nullable = true)
-- level: string (nullable = true)
-- location: string (nullable = true)
-- method: string (nullable = true)
-- page: string (nullable = true)
-- registration: double (nullable = true)
-- sessionId: long (nullable = true)
-- status: long (nullable = true)
-- ts: long (nullable = true)
-- userAgent: string (nullable = true)
-- userId: string (nullable = true)
-- video: string (nullable = true)
-- youtuber: string (nullable = true)

In [5]:
user_log.show(n=1)

+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-----------------+---------+------+-------------+--------------------+------+-----------------+-------------+
 auth|firstName|gender|itemInSession|lastName| length|level| location|method| page| registration|sessionId|status| ts| userAgent|userId| video| youtuber|
+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-----------------+---------+------+-------------+--------------------+------+-----------------+-------------+
Logged In| Jayden| M| 0| Bell|182.85669| free|Dallas-Fort Worth...| PUT|NextVideo|1.540991795796E12| 829| 200|1543537327796|Mozilla/5.0 (comp...| 91|Jim Henson's Dead|Stephen Lynch|
+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-----------------+---------+------+-------------+--------------------+------+-----------------+-------------+
only showing top 1 row

In [6]:
user_log.describe().show()

+-------+----------+---------+------+------------------+--------+------------------+-----+--------------------+------+-------+--------------------+------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+
summary| auth|firstName|gender| itemInSession|lastName| length|level| location|method| page| registration| sessionId| status| ts| userAgent| userId| video| youtuber|
+-------+----------+---------+------+------------------+--------+------------------+-----+--------------------+------+-------+--------------------+------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+
 count| 2574| 2451| 2451| 2574| 2451| 2160| 2574| 2451| 2574| 2574| 2451| 2574| 2574| 2574| 2451| 2574| 2160| 2160|
 mean| null| null| null|19.690365190365192| null|248.25115654629641| null| null| null| null|1.540800438633209...| 890.6087801087801|203.31779331779333|1.543298890969271...| null| 56.82578539371685| 2042.5| 311.0|
 stddev| null| null| null|21.382841073290823| null|110.83686954974532| null| null| null| null|2.3906306749278748E8|189.86547187993872|18.960777781496798|2.1934524600681496E8| null|27.071926948771846| 3115.502511634359| 0.0|
 min| Logged In| Adler| F| 0|Arellano| 22.12526| free|Atlanta-Sandy Spr...| GET| About| 1.540006905796E12| 21| 200| 1542845032796|"Mozilla/5.0 (Mac...| | I Will Not Reap ...|'N Sync/Phil Collins|
 max|Logged Out| Zachary| M| 117| Young| 2594.87302| paid| Yuba City, CA| PUT|Upgrade| 1.541098488796E12| 1114| 404| 1543607664796|Mozilla/5.0 (comp...| 98|the king of wishf...|ÃÂngeles del In...|
+-------+----------+---------+------+------------------+--------+------------------+-----+--------------------+------+-------+--------------------+------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+

In [7]:
user_log.describe("Youtuber").show()

+-------+--------------------+
summary| Youtuber|
+-------+--------------------+
 count| 2160|
 mean| 311.0|
 stddev| 0.0|
 min|'N Sync/Phil Collins|
 max|ÃÂngeles del In...|
+-------+--------------------+

In [8]:
user_log.describe("sessionId").show()

+-------+------------------+
summary| sessionId|
+-------+------------------+
 count| 2574|
 mean| 890.6087801087801|
 stddev|189.86547187993872|
 min| 21|
 max| 1114|
+-------+------------------+

In [9]:
user_log.count()

Out[30]: 2574

In [10]:
user_log.select("page").dropDuplicates().sort("page").show()

+-------------+
 page|
+-------------+
 About|
 Downgrade|
 Error|
 Help|
 Home|
 Login|
 Logout|
 NextVideo|
Save Settings|
 Settings|
 Upgrade|
+-------------+

In [11]:
user_log.select(["userId", "firstname", "page", "video"]).where(user_log.userId == "73").collect()

Out[36]: [Row(userId='73', firstname='Jacob', page='NextVideo', video='Shell Shock'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Women Lose Weight (Feat: Slick Rick)'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video="Won't Go Home Without You"),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Hey_ Soul Sister'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video="I'm In Miami Bitch"),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Sexy Bitch'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='The Cure & The Cause (Dennis Ferrer Remix)'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Staring At Me'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='The Second You Sleep'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Wide Eyes'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='La Resistance (Medley) (LP Version)'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Restless'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Noc...'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Like You'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='God Put A Smile Upon Your Face'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Playboy'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Every Time You Go'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Natural Anthem (Album)'),
 Row(userId='73', firstname='Jacob', page='Logout', video=None),
 Row(userId='73', firstname='Jacob', page='NextVideo', video="Ain't No Sunshine"),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='The Girl and The Robot'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='The Great Satan (What Would Satan Do Mix)'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='The Screamer'),
 Row(userId='73', firstname='Jacob', page='Home', video=None),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Dirty Little Secret'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='The Mighty Ride Of The Firelord'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Think of Me'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Crack A Bottle'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Grits'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Take It There'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Tribulations'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video="Bella's Lullaby (Twilight Soundtrack Version)"),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Rat Fink'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Inaudible Melodies'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Where The Wild Things Were'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Last Night On Earth [feat. Green Day & The Cast Of American Idiot] (Album Version)'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Witch Doctor'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video="Travelin' Band"),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Love Resurrection'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Rocker'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Over Now'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Cover Me'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Sehr kosmisch'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='We throw parties_ you throw knives'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='One Less Lonely Girl'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Doperide'),
 Row(userId='73', firstname='Jacob', page='NextVideo', video='Why Georgia'),
 Row(userId='73', firstname='Jaco

In [12]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)

In [13]:
user_log = user_log.withColumn("hour", get_hour(user_log.ts))

In [14]:
user_log.show(n=2)

+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-----------------+---------+------+-------------+--------------------+------+-----------------+-------------+----+
 auth|firstName|gender|itemInSession|lastName| length|level| location|method| page| registration|sessionId|status| ts| userAgent|userId| video| youtuber|hour|
+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-----------------+---------+------+-------------+--------------------+------+-----------------+-------------+----+
Logged In| Jayden| M| 0| Bell|182.85669| free|Dallas-Fort Worth...| PUT|NextVideo|1.540991795796E12| 829| 200|1543537327796|Mozilla/5.0 (comp...| 91|Jim Henson's Dead|Stephen Lynch| 0|
Logged In| Jacob| M| 0| Klein| 247.562| paid|Tampa-St. Petersb...| PUT|NextVideo|1.540558108796E12| 1049| 200|1543540121796|"Mozilla/5.0 (Mac...| 73| Shell Shock| Manowar| 1|
+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-----------------+---------+------+-------------+--------------------+------+-----------------+-------------+----+
only showing top 2 rows

In [15]:
videos_in_hour = user_log.filter(user_log.page == "NextVideo").groupby(user_log.hour).count().orderBy(user_log.hour.cast("float"))

In [16]:
videos_in_hour.show()

+----+-----+
hour|count|
+----+-----+
 0| 59|
 1| 53|
 2| 25|
 3| 39|
 4| 58|
 5| 57|
 6| 68|
 7| 59|
 8| 50|
 9| 35|
 10| 49|
 11| 98|
 12| 143|
 13| 102|
 14| 153|
 15| 182|
 16| 191|
 17| 184|
 18| 147|
 19| 92|
+----+-----+
only showing top 20 rows

In [17]:
videos_in_hour_pd = videos_in_hour.toPandas()
videos_in_hour_pd.hour = pd.to_numeric(videos_in_hour_pd.hour)

In [18]:
plt.scatter(videos_in_hour_pd["hour"], videos_in_hour_pd["count"])
plt.xlim(-1, 24);
plt.ylim(0, 1.2 * max(videos_in_hour_pd["count"]))
plt.xlabel("Hour")
plt.ylabel("Videos played");